# AFT第一次内培

## 本地使用数据

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
# import optiver2023 # 个人理解这个包是在比赛进行中能够实时给予新的验证数据？用于读取测试数据和提交预测。


In [16]:
# 读取数据
Xy = pd.read_csv('train.csv')

## 了解数据

In [5]:
Xy.head(10)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4
5,5,0,0,0.00,0,1.000635,13552875.92,NaN,NaN,0.999779,1962.72,1.000635,5647.65,1.0,6.779432,0,0_0_5
6,6,0,0,969969.40,1,1.000115,3647503.98,NaN,NaN,0.999506,6663.16,1.000283,3810.48,1.0,-2.499819,0,0_0_6
7,7,0,0,9412959.10,1,0.999818,21261245.87,NaN,NaN,0.999741,5139.20,1.000130,2570.60,1.0,-1.959801,0,0_0_7
8,8,0,0,2394875.85,1,0.999916,9473209.08,NaN,NaN,0.999022,52011.60,1.000041,2169.36,1.0,-5.970001,0,0_0_8
9,9,0,0,3039700.65,-1,1.000969,6248958.45,NaN,NaN,0.999354,6191.00,1.000646,6199.00,1.0,7.970333,0,0_0_9


In [ ]:
def inspect_columns(df):
    # A helper function that does a better job than df.info() and df.describe()
    result = pd.DataFrame({
        'unique': df.nunique() == len(df),
        'cardinality': df.nunique(),
        'with_null': df.isna().any(),
        'null_pct': round((df.isnull().sum() / len(df)) * 100, 2),
        '1st_row': df.iloc[0],
        'random_row': df.iloc[np.random.randint(low=0, high=len(df))],
        'last_row': df.iloc[-1],
        'dtype': df.dtypes
    })
    return result

inspect_columns(Xy)

In [ ]:
Xy.describe()

In [ ]:
# 检查逻辑异常
logic_errors = Xy[Xy['ask_price'] < Xy['bid_price']]
logic_errors

In [19]:
import dask.dataframe as dd

Xy_dask = dd.from_pandas(Xy, npartitions=10)

Q1_far_price = Xy_dask['far_price'].quantile(0.25).compute()
Q3_far_price = Xy_dask['far_price'].quantile(0.75).compute()
IQR_far_price = Q3_far_price - Q1_far_price

far_price_lower_bound = Q1_far_price - 1.5 * IQR_far_price
far_price_upper_bound = Q3_far_price + 1.5 * IQR_far_price

outliers_far_price = Xy_dask[(Xy_dask['far_price'] < far_price_lower_bound) | (Xy_dask['far_price'] > far_price_upper_bound)].compute()
outliers_far_price


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
5731,1,0,300,242332.96,-1,1.000073,3242054.27,0.981974,0.994490,0.999544,43205.40,1.000308,2042.76,1.000273,-1.329780,30,0_300_1
5736,6,0,300,2290880.11,1,1.000493,6222660.84,1.020254,1.008032,1.000010,110949.94,1.000493,476.41,1.000490,-2.139807,30,0_300_6
5737,7,0,300,5743260.87,-1,0.997609,59880787.67,0.972086,0.992924,0.997609,6410.30,0.998138,17958.36,0.997748,11.379719,30,0_300_7
5738,8,0,300,3049995.87,1,0.999183,21575575.33,1.016710,1.005503,0.998155,6495.81,1.000248,299433.78,0.998200,0.729561,30,0_300_8
5740,10,0,300,1045511.01,-1,1.001055,21776715.62,0.986271,0.994461,1.000896,39335.89,1.001109,4518.00,1.001087,1.150370,30,0_300_10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5236440,60,480,470,23060600.51,-1,0.991454,82672604.11,0.971640,0.983870,0.991113,21759.00,0.991796,46886.68,0.991329,-7.060170,26447,480_470_60
5236478,98,480,470,1124978.74,-1,0.995065,4810987.03,0.985477,0.992934,0.995065,247323.20,0.996130,424003.80,0.995458,-10.030270,26447,480_470_98
5236505,125,480,470,2264360.77,-1,0.995766,9507583.12,0.976114,0.990683,0.995721,108817.44,0.996175,140473.60,0.995919,0.020266,26447,480_470_125
5236558,178,480,470,5196592.84,-1,0.996325,13840548.16,0.977046,0.993381,0.996177,49481.39,0.996325,5348.30,0.996310,-4.299879,26447,480_470_178


## 基础数据处理

In [17]:
# 获取 target 不为空的行，并删除 'row_id' 和 'time_id' 列
X_train = Xy.query("target.notna()").drop(['row_id', 'time_id'], axis=1)

# 分离特征和目标变量
y_train = X_train.pop("target")

In [19]:
# 处理缺失值
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
imputer = SimpleImputer(strategy='mean')
X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])

## 简单的特征工程

In [20]:
def calculate_imbalance_features(df):
    # 买卖双方的力量差距
    df['imb_s1'] = df.eval('(bid_size - ask_size) / (bid_size + ask_size)')  

    # 已匹配订单和未匹配订单之间的差异
    df['imb_s2'] = df.eval('(imbalance_size - matched_size) / (matched_size + imbalance_size)') 

    return df

def liquidity_feature(df):
    # 市场流动性特征
    df['liquidity'] = df.eval('(bid_size + ask_size)')  

# 应用特征工程
X_train = calculate_imbalance_features(X_train)
X_train = liquidity_feature(X_train)

## xgboost模型训练

In [10]:
# 初始化XGBoost回归模型，后续可加入调参
xgb_model = xgb.XGBRegressor(
    n_estimators=100,            # 树的数量
    max_depth=6,                 # 树的深度
    learning_rate=0.1,           # 学习率
    objective='reg:squarederror', # 使用平方误差作为目标函数
    subsample=0.8,               # 每棵树使用的数据比例
    colsample_bytree=0.8,        # 每棵树使用的特征比例
    n_jobs=-1
)

In [11]:
# 使用时间序列分割器
tscv = TimeSeriesSplit(n_splits=5)

val_scores = []
for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train)):
    X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
    X_val_fold, y_val_fold = X_train.iloc[val_idx], y_train.iloc[val_idx]
    
    # 训练XGBoost模型
    xgb_model.fit(X_train_fold, y_train_fold)
    
    # 验证模型
    y_val_pred = xgb_model.predict(X_val_fold)
    val_mae = mean_absolute_error(y_val_fold, y_val_pred)
    val_scores.append(val_mae)
    
    print(f'Fold {fold+1}, MAE: {val_mae:.4f}')

# 输出验证得分的平均值和标准差
print(f'Val Score: {np.mean(val_scores):.4f} ± {np.std(val_scores):.4f}')

Fold 1, MAE: 7.5705
Fold 2, MAE: 6.8214
Fold 3, MAE: 6.1691
Fold 4, MAE: 6.3849
Fold 5, MAE: 5.9297
Val Score: 6.5751 ± 0.5778


## 评估

In [14]:
split_index = int(len(X_train) * (1 - 0.2))
X_train, X_test = X_train[:split_index], X_train[split_index:]
y_train, y_test = y_train[:split_index], y_train[split_index:]

# 使用完整的训练数据重新训练模型
xgb_model.fit(X_train, y_train)

y_val_pred = xgb_model.predict(X_test)
val_mae = mean_absolute_error(y_test, y_val_pred)

val_mae

6.067381597311286